In [1]:
import torch
from torch import nn
import numpy as np
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [22]:
df = pd.read_csv("Student_Performance.csv") # Reading and saving dataframe
features = ["Hours Studied", "Previous Scores", "Extracurricular Activities", "Sleep Hours", "Sample Question Papers Practiced"]
target = "Performance Index"

df["Extracurricular Activities"] = df["Extracurricular Activities"].map({'Yes': 1, 'No': 0})
X = df[features].values
y = df[target].values.reshape(len(df[target].values), 1) # Equivalent y = df[target].values.reshape(-1, 1)
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()
print("Features: ", X)
print("Features size: ", X.shape)
print()
print("Target: ", y)
print("Target size: ", y.shape)

Features:  tensor([[ 7., 99.,  1.,  9.,  1.],
        [ 4., 82.,  0.,  4.,  2.],
        [ 8., 51.,  1.,  7.,  2.],
        ...,
        [ 6., 83.,  1.,  8.,  5.],
        [ 9., 97.,  1.,  7.,  0.],
        [ 7., 74.,  0.,  8.,  1.]])
Features size:  torch.Size([10000, 5])

Target:  tensor([[91.],
        [65.],
        [45.],
        ...,
        [74.],
        [95.],
        [64.]])
Target size:  torch.Size([10000, 1])


In [23]:
train_split = int(len(X) * 0.8)
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]
print((len(X_train), len(y_train), len(X_test), len(y_test)))

(8000, 8000, 2000, 2000)


In [24]:
class LinearRegression(nn.Module):
  def __init__(self):
    super().__init__()
    self.w1 = nn.Parameter(torch.randn(1,
                                    requires_grad=True,
                                    dtype=torch.float))
    self.w2 = nn.Parameter(torch.randn(1,
                                    requires_grad=True,
                                    dtype=torch.float))
    self.w3 = nn.Parameter(torch.randn(1,
                                    requires_grad=True,
                                    dtype=torch.float))
    self.w4 = nn.Parameter(torch.randn(1,
                                    requires_grad=True,
                                    dtype=torch.float))
    self.w5 = nn.Parameter(torch.randn(1,
                                    requires_grad=True,
                                    dtype=torch.float))
    self.bias = nn.Parameter(torch.randn(1,
                                    requires_grad=True,
                                    dtype=torch.float))
  def forward(self, hrs, prevScores, extraCurry, sleepHrs, sampleQp):
    return hrs * self.w1 + prevScores * self.w2 + extraCurry * self.w3 + sleepHrs * self.w4 + sampleQp * self.w5 + self.bias

torch.manual_seed(42)
model_0 = LinearRegression()
model_0.state_dict()

OrderedDict([('w1', tensor([0.3367])),
             ('w2', tensor([0.1288])),
             ('w3', tensor([0.2345])),
             ('w4', tensor([0.2303])),
             ('w5', tensor([-1.1229])),
             ('bias', tensor([-0.1863]))])

In [19]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(params=model_0.parameters(),
                            lr=0.01)


In [20]:
epochs = 500
for epoch in range(epochs):
  model_0.train()
  # 1. Forward pass
  train_preds = model_0.forward(X_train[:, 0], X_train[:, 1], X_train[:, 2], X_train[:, 3], X_train[:, 4])

  # 2. Compute loss
  train_loss = loss_fn(train_preds, y_train)

  # 3. Optimizer zero grad
  optimizer.zero_grad()

  # 4. loss backward
  train_loss.backward()

  # 5. Optimizer step
  optimizer.step()

  model_0.eval()

  with torch.inference_mode():
    test_preds = model_0.forward(X_test[:, 0], X_test[:, 1], X_test[:, 2], X_test[:, 3], X_test[:, 4])
    test_loss = loss_fn(test_preds, y_test)

  if epoch % 10 == 0:
    print(f"Epoch: {epoch} | Train loss: {train_loss} | Test loss: {test_loss}")

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([8000, 1])) that is different to the input size (torch.Size([8000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([2000, 1])) that is different to the input size (torch.Size([2000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0 | Train loss: 3061.93408203125 | Test loss: 3065.53759765625
Epoch: 10 | Train loss: 2883.225341796875 | Test loss: 2885.189208984375
Epoch: 20 | Train loss: 2712.14208984375 | Test loss: 2712.671630859375
Epoch: 30 | Train loss: 2549.40771484375 | Test loss: 2548.683349609375
Epoch: 40 | Train loss: 2395.379638671875 | Test loss: 2393.5498046875
Epoch: 50 | Train loss: 2250.114990234375 | Test loss: 2247.304931640625
Epoch: 60 | Train loss: 2113.47998046875 | Test loss: 2109.797119140625
Epoch: 70 | Train loss: 1985.2340087890625 | Test loss: 1980.7752685546875
Epoch: 80 | Train loss: 1865.086181640625 | Test loss: 1859.9398193359375
Epoch: 90 | Train loss: 1752.724365234375 | Test loss: 1746.971923828125
Epoch: 100 | Train loss: 1647.825927734375 | Test loss: 1641.544189453125
Epoch: 110 | Train loss: 1550.068115234375 | Test loss: 1543.3277587890625
Epoch: 120 | Train loss: 1459.1275634765625 | Test loss: 1451.994140625
Epoch: 130 | Train loss: 1374.6834716796875 | Test los

KeyboardInterrupt: 

In [25]:
with torch.inference_mode():
  print(model_0.forward(torch.tensor(7), torch.tensor(99), torch.tensor(1), torch.tensor(9), torch.tensor(1)))

tensor([16.1072])


In [39]:
class LinearRegressionUpd(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(in_features=5, out_features=1)

  def forward(self, X: torch.tensor):
    return self.linear(X)

In [40]:
torch.manual_seed(42)
model_1 = LinearRegressionUpd()
model_1.state_dict()

OrderedDict([('linear.weight',
              tensor([[ 0.3419,  0.3712, -0.1048,  0.4108, -0.0980]])),
             ('linear.bias', tensor([0.0902]))])

In [41]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(params=model_1.parameters(),
                             lr=0.01)

In [56]:
epochs = 1000
for epoch in range(epochs):
  model_1.train()
  # 1. Forward pass
  train_preds = model_1(X_train)
  # 2. Compute loss
  train_loss = loss_fn(train_preds, y_train)

  # 3. Zero gradient
  optimizer.zero_grad()

  # 4. Backward propagation
  train_loss.backward()

  # 5. Optimizer step
  optimizer.step()

  model_1.eval()
  with torch.inference_mode():
    test_preds = model_1(X_test)
    test_loss = loss_fn(test_preds, y_test)
  if epoch % 10 == 0:
    print(f"Epoch: {epoch} | Train loss: {train_loss} | Test loss: {test_loss}")

Epoch: 0 | Train loss: 4.128206253051758 | Test loss: 4.244961261749268
Epoch: 10 | Train loss: 4.1378583908081055 | Test loss: 4.241723537445068
Epoch: 20 | Train loss: 4.133723735809326 | Test loss: 4.261905193328857
Epoch: 30 | Train loss: 4.128113746643066 | Test loss: 4.252127170562744
Epoch: 40 | Train loss: 4.128567695617676 | Test loss: 4.2453107833862305
Epoch: 50 | Train loss: 4.128189563751221 | Test loss: 4.245049953460693
Epoch: 60 | Train loss: 4.128085136413574 | Test loss: 4.2473931312561035
Epoch: 70 | Train loss: 4.128075122833252 | Test loss: 4.248076438903809
Epoch: 80 | Train loss: 4.128062725067139 | Test loss: 4.246922016143799
Epoch: 90 | Train loss: 4.128054618835449 | Test loss: 4.2470197677612305
Epoch: 100 | Train loss: 4.128055572509766 | Test loss: 4.247386932373047
Epoch: 110 | Train loss: 4.128055095672607 | Test loss: 4.24706506729126
Epoch: 120 | Train loss: 4.128055095672607 | Test loss: 4.247253894805908
Epoch: 130 | Train loss: 4.128054618835449 | T

In [61]:
from tabulate import tabulate
final_pred = model_1(X).detach().numpy()
table2 = np.column_stack((y,final_pred))

print(tabulate(table2[:5], headers=("Actual_Data","Data_Predicted"), tablefmt="fancy_grid"))

╒═══════════════╤══════════════════╕
│   Actual_Data │   Data_Predicted │
╞═══════════════╪══════════════════╡
│            91 │          91.8244 │
├───────────────┼──────────────────┤
│            65 │          63.1528 │
├───────────────┼──────────────────┤
│            45 │          45.0539 │
├───────────────┼──────────────────┤
│            36 │          36.5691 │
├───────────────┼──────────────────┤
│            66 │          67.052  │
╘═══════════════╧══════════════════╛
